# Analyzing results from hyperparameter tuning
In this example we'll run a hyperparameter tuning experiment and analyze the results.

What we'll cover:
- How to load data from an Sklearn example dataset
- How to initialize an XGBoost trainer
- How to define a search space for regular XGBoost parameters and for data preprocessors
- How to fetch the best obtained result from the tuning run
- How to fetch a dataframe to do further analysis on the results

First we start with some imports. We'll use the [Covertype dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn-datasets-fetch-covtype) provided from sklearn to train a multiclass classification task using XGBoost.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_covtype

import ray
from ray import tune
from ray.ml import RunConfig
from ray.ml.preprocessors import StandardScaler
from ray.ml.train.integrations.xgboost import XGBoostTrainer
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

We'll define a utility function to create a Ray dataset from the Sklearn dataset. We expect the target column to be in the dataframe, so we'll add it to the dataframe manually.
We don't do any test/validation splitting here - pelase check out our other examples for this.

In [2]:
def get_training_data() -> ray.data.Dataset:
    data_raw = fetch_covtype()
    df = pd.DataFrame(data_raw["data"], columns=data_raw["feature_names"])
    df["target"] = data_raw["target"]
    return ray.data.from_pandas(df)


train_dataset = get_training_data()

2022-05-09 15:10:00,737	INFO services.py:1484 -- View the Ray dashboard at http://127.0.0.1:8265


Let's take a look at the schema here:

In [3]:
print(train_dataset)

Dataset(num_blocks=1, num_rows=581012, schema={Elevation: float64, Aspect: float64, Slope: float64, Horizontal_Distance_To_Hydrology: float64, Vertical_Distance_To_Hydrology: float64, Horizontal_Distance_To_Roadways: float64, Hillshade_9am: float64, Hillshade_Noon: float64, Hillshade_3pm: float64, Horizontal_Distance_To_Fire_Points: float64, Wilderness_Area_0: float64, Wilderness_Area_1: float64, Wilderness_Area_2: float64, Wilderness_Area_3: float64, Soil_Type_0: float64, Soil_Type_1: float64, Soil_Type_2: float64, Soil_Type_3: float64, Soil_Type_4: float64, Soil_Type_5: float64, Soil_Type_6: float64, Soil_Type_7: float64, Soil_Type_8: float64, Soil_Type_9: float64, Soil_Type_10: float64, Soil_Type_11: float64, Soil_Type_12: float64, Soil_Type_13: float64, Soil_Type_14: float64, Soil_Type_15: float64, Soil_Type_16: float64, Soil_Type_17: float64, Soil_Type_18: float64, Soil_Type_19: float64, Soil_Type_20: float64, Soil_Type_21: float64, Soil_Type_22: float64, Soil_Type_23: float64, So

Ray AIR allows us to define and tune preprocessors. For instance, we may want to decide if we should scale the "Wilderness area" parameters or the "Soil type" parameters. For both, we'll just try standard scalers:

In [4]:
# Let's tune the preprocessors
prep_v1 = StandardScaler(columns=[f"Wilderness_Area_{i}" for i in range(4)])
prep_v2 = StandardScaler(columns=[f"Soil_Type_{i}" for i in range(40)])

Since we'll be training a multiclass prediction model, we have to pass some information to XGBoost. For instance, XGBoost expects us to provide the number of classes, and multiclass-enabled evaluation metrices.

With these parameters in place, we'll create a Ray AIR `XGBoostTrainer`:

In [5]:
# XGBoost specific params
params = {
    "tree_method": "approx",
    "objective": "multi:softmax",
    "eval_metric": ["mlogloss", "merror"],
    "num_class": 8,
}

trainer = XGBoostTrainer(
    scaling_config={"num_workers": 2},
    label_column="target",
    params=params,
    datasets={"train": train_dataset},
    num_boost_round=10,
)

The trainer contains a few default parameters. We can now create the Tuner with a search space for other parameters. During the run, it will then overwrite the default parameters in the trainer according to the search specification.

Here, we just want to tune the XGBoost `max_depth` parameter. Additionally, we want to compare our two `preprocessor` pipelines. Lastly, we configure Tune to minimize the `train-mlogloss` metric. In random search, this doesn't affect the evaluated configurations, but it will affect our default results fetching for analysis later.

In [6]:
tuner = Tuner(
    trainer,
    run_config=RunConfig(verbose=3),
    param_space={
        "params": {"max_depth": tune.randint(2, 8)},
        "preprocessor": tune.grid_search([prep_v1, prep_v2]),
    },
    tune_config=TuneConfig(num_samples=4, metric="train-mlogloss", mode="min"),
)

Let's run the tuning. This will take a few minutes to complete.

In [7]:
results = tuner.fit()

Trial name,status,loc,params/max_depth,preprocessor,iter,total time (s),train-mlogloss,train-merror
XGBoostTrainer_57a6f_00000,TERMINATED,127.0.0.1:40794,2,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,87.1825,0.79498,0.311035
XGBoostTrainer_57a6f_00001,TERMINATED,127.0.0.1:40800,7,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,167.86,0.559286,0.197125
XGBoostTrainer_57a6f_00002,TERMINATED,127.0.0.1:40801,3,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,111.875,0.742797,0.279679
XGBoostTrainer_57a6f_00003,TERMINATED,127.0.0.1:40802,7,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,168.114,0.559286,0.197125
XGBoostTrainer_57a6f_00004,TERMINATED,127.0.0.1:40803,4,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,115.855,0.69843,0.262468
XGBoostTrainer_57a6f_00005,TERMINATED,127.0.0.1:40987,3,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,84.7425,0.742797,0.279679
XGBoostTrainer_57a6f_00006,TERMINATED,127.0.0.1:41030,6,"StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)",10,102.815,0.607805,0.220687
XGBoostTrainer_57a6f_00007,TERMINATED,127.0.0.1:41046,6,"StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, stats=None)",10,97.4885,0.607805,

(GBDTTrainable pid=40801) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=40794) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=40803) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(GBDTTrainable pid=40794) 2022-05-09 15:10:22,642	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(GBDTTrainable pid=40803) 2022-05-09 15:10:23,136	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(GBDTTrainable pid=40800) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartiti

Result for XGBoostTrainer_57a6f_00000:
  date: 2022-05-09_15-10-38
  done: false
  experiment_id: db2d456d77f54ce38738a44e228f27ea
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 40794
  should_checkpoint: true
  time_since_restore: 29.05068802833557
  time_this_iter_s: 29.05068802833557
  time_total_s: 29.05068802833557
  timestamp: 1652101838
  timesteps_since_restore: 0
  train-merror: 0.333787
  train-mlogloss: 1.568608
  training_iteration: 1
  trial_id: 57a6f_00000
  warmup_time: 0.003194093704223633
  


(GBDTTrainable pid=40800) 2022-05-09 15:10:38,794	INFO main.py:984 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(GBDTTrainable pid=40803) 2022-05-09 15:10:39,035	INFO main.py:1029 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=40857) [15:10:39] task [xgboost.ray]:4560524880 got new rank 1
(_RemoteRayXGBoostActor pid=40856) [15:10:39] task [xgboost.ray]:4577444688 got new rank 0


Result for XGBoostTrainer_57a6f_00000:
  date: 2022-05-09_15-10-48
  done: false
  experiment_id: db2d456d77f54ce38738a44e228f27ea
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 2
  node_ip: 127.0.0.1
  pid: 40794
  should_checkpoint: true
  time_since_restore: 39.13641595840454
  time_this_iter_s: 10.08572793006897
  time_total_s: 39.13641595840454
  timestamp: 1652101848
  timesteps_since_restore: 0
  train-merror: 0.330764
  train-mlogloss: 1.330721
  training_iteration: 2
  trial_id: 57a6f_00000
  warmup_time: 0.003194093704223633
  


...

(GBDTTrainable pid=41030) 2022-05-09 15:13:55,598	INFO main.py:1526 -- [RayXGBoost] Finished XGBoost training on training data with total N=581,012 in 88.82 seconds (78.35 pure XGBoost training time).
2022-05-09 15:13:56,475	INFO logger.py:730 -- Removed the following hyperparameter values when logging to tensorboard: {'preprocessor': StandardScaler(columns=['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3'], ddof=0, stats=None)}


Result for XGBoostTrainer_57a6f_00006:
  date: 2022-05-09_15-13-55
  done: true
  experiment_id: d0e06dc21aba4a7ab3a705f32d2c5f0d
  experiment_tag: 6_max_depth=6,preprocessor=StandardScaler_columns_Wilderness_Area_0_Wilderness_Area_1_Wilderness_Area_2_Wilderness_Area_3_ddof_0_stats_None
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 41030
  should_checkpoint: true
  time_since_restore: 102.81524395942688
  time_this_iter_s: 6.15753698348999
  time_total_s: 102.81524395942688
  timestamp: 1652102035
  timesteps_since_restore: 0
  train-merror: 0.220687
  train-mlogloss: 0.607805
  training_iteration: 10
  trial_id: 57a6f_00006
  warmup_time: 0.005061149597167969
  
Result for XGBoostTrainer_57a6f_00007:
  date: 2022-05-09_15-13-56
  done: false
  experiment_id: 0e0fb2a0b9d64fcaa5d7f90245c46f17
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 41046
  should_checkpoint: true
  time_since_restore: 9

(GBDTTrainable pid=41046) 2022-05-09 15:13:56,624	INFO main.py:1526 -- [RayXGBoost] Finished XGBoost training on training data with total N=581,012 in 89.04 seconds (78.12 pure XGBoost training time).
2022-05-09 15:13:57,293	INFO logger.py:730 -- Removed the following hyperparameter values when logging to tensorboard: {'preprocessor': StandardScaler(columns=['Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39'], ddof=0, 

Result for XGBoostTrainer_57a6f_00007:
  date: 2022-05-09_15-13-56
  done: true
  experiment_id: 0e0fb2a0b9d64fcaa5d7f90245c46f17
  experiment_tag: 7_max_depth=6,preprocessor=StandardScaler_columns_Soil_Type_0_Soil_Type_1_Soil_Type_2_Soil_Type_3_Soil_Type_4_Soil_Type_5_Soil_Type_6_Soil_Type_7_Soil_Type_8_Soil_Type_9_Soil_Type_10_Soil_Type_11_Soil_Type_12_Soil_Type_13_Soil_Type_14_Soil_Type_15_Soil_Type_16_Soil_Type_17_Soil_Type_18_Soil_Type_19_Soil_Type_20_Soil_Type_21_Soil_Type_22_Soil_Type_23_Soil_Type_24_Soil_Type_25_Soil_Type_26_Soil_Type_27_Soil_Type_28_Soil_Type_29_Soil_Type_30_Soil_Type_31_Soil_Type_32_Soil_Type_33_Soil_Type_34_Soil_Type_35_Soil_Type_36_Soil_Type_37_Soil_Type_38_Soil_Type_39_ddof_0_stats_None
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 41046
  should_checkpoint: true
  time_since_restore: 97.48850464820862
  time_this_iter_s: 5.544768571853638
  time_total_s: 97.48850464820862
  timestamp: 1652102036
  timesteps_

2022-05-09 15:13:57,554	INFO tune.py:753 -- Total run time: 232.65 seconds (231.22 seconds for the tuning loop).


Now that we obtained the results, we can analyze them. For instance, we can fetch the best observed result according to the configured `metric` and `mode` and print it:

In [8]:
# This will fetch the best result according to the `metric` and `mode` specified
# in the `TuneConfig` above:

best_result = results.get_best_result()

print("Best result error rate", best_result.metrics["train-merror"])

Best result error rate 0.197125


For more sophisticated analysis, we can get a pandas dataframe with all trial results:

In [9]:
df = results.get_dataframe()
print(df.columns)

Index(['train-mlogloss', 'train-merror', 'time_this_iter_s',
       'should_checkpoint', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'trial_id', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore', 'warmup_time',
       'config/params', 'config/preprocessor', 'logdir'],
      dtype='object')


As an example, let's group the results per preprocessor and fetch the minimal obtained values:

In [10]:
groups = df.groupby("config/preprocessor")
mins = groups.min()

print("Preprocessor 1 minimum log loss:", mins.loc[str(prep_v1)]["train-mlogloss"])
print("Preprocessor 2 minimum log loss:", mins.loc[str(prep_v2)]["train-mlogloss"])

print("Preprocessor 1 minimum error:", mins.loc[str(prep_v1)]["train-merror"])
print("Preprocessor 2 minimum error:", mins.loc[str(prep_v2)]["train-merror"])

Preprocessor 1 minimum log loss: 0.607805
Preprocessor 2 minimum log loss: 0.559286
Preprocessor 1 minimum error: 0.220687
Preprocessor 2 minimum error: 0.197125


The `results.get_dataframe()` returns the last reported results per trial. If you want to obtain the best _ever_ observed results, you can pass the `filter_metric` and `filter_mode` arguments to `results.get_dataframe()`. In our example, we'll filter the minimum _ever_ observed `train-merror` for each trial:

In [11]:
df_min_error = results.get_dataframe(filter_metric="train-merror", filter_mode="min")
df_min_error["train-merror"]

0    0.310307
1    0.197125
2    0.279679
3    0.197125
4    0.262468
5    0.279679
6    0.220687
7    0.220687
Name: train-merror, dtype: float64

And that's how you analyze your hyperparameter tuning results. If you would like to have access to more analytics, please feel free to file a feature request e.g. [as a Github issue](https://github.com/ray-project/ray/issues) or on our [Discuss platform](https://discuss.ray.io/)!